In [ ]:
import pandas as pd
import ast
import re

In [ ]:
df = pd.read_csv("../data/final_df.csv")

In [ ]:
def clean_basic_description(df):
    ## Remove the word "Entire" and "in Calgary, Canada" from the basic_description column
    df["basic_description"] = df["basic_description"].apply(
        lambda x: x.replace("Entire", "").replace("in Calgary, Canada", "")
    )
    ## Remvoving trailing and leading whitespaces
    df["basic_description"] = df["basic_description"].str.strip()
    return df


def clean_prices(df):
    df["prices"] = df["prices"].apply(
        lambda x: x.replace("\nnight", "")
        .replace("\n", "*/")
        .replace("$", "")
        .replace(" CAD", "")
    )
    df["prices"] = df["prices"].str.strip()
    ## split the prices column into original_price and discounted_price
    df["discounted_price"] = df["prices"].apply(
        lambda x: x.split("*/")[1] if "*/" in x else x
    )
    df["original_price"] = df["prices"].apply(lambda x: x.split("*/")[0])
    ## Remove the prices column
    df = df.drop("prices", axis=1)
    return df


def clean_ratings(df):
    df["ratings"] = df["ratings"].fillna("\nNew")
    df["ratings"] = df["ratings"].apply(lambda x: x.split("\n")[1])
    ## Remove the word "New" from the ratings column and replace with 0.0 (0)
    ## combining these steps into one line
    df["ratings"] = df["ratings"].str.replace("New", "0.0 (0)", regex=False)
    df["ratings"] = df["ratings"].str.replace(" (", "*/", regex=False)
    df["ratings"] = df["ratings"].str.replace(")", "", regex=False)
    ## split the ratings column into average_rating and num_reviews
    df["average_rating"] = df["ratings"].apply(lambda x: x.split("*/")[0])
    df["num_reviews"] = df["ratings"].apply(lambda x: x.split("*/")[1])
    ## Remove the ratings column
    df = df.drop("ratings", axis=1)
    return df


def clean_basic_info(df):
    df["basic_info"] = df["basic_info"].apply(
        lambda x: x.replace("bedrooms", "bedroom")
        .replace("beds", "bed")
        .replace("bedroom", "room")
    )

    ## split the basic_info column into guests, bedrooms and beds
    def extract_details(details):
        if not isinstance(details, str):
            details = ""

        # Extract numbers using regex or assign default values if not found
        guests_match = re.search(r"(\d+)\s*guests", details)
        bedrooms_match = re.search(r"(\d+)\s*room", details)
        beds_match = re.search(r"(\d+)\s*bed", details)

        guests = int(guests_match.group(1)) if guests_match else 2
        bedrooms = int(bedrooms_match.group(1)) if bedrooms_match else 1
        beds = int(beds_match.group(1)) if beds_match else 1

        return pd.Series([guests, bedrooms, beds])

    ## Extract the details and assign to new columns
    df[["Guests", "Bedrooms", "Beds"]] = df["basic_info"].apply(extract_details)
    df = df.drop("basic_info", axis=1)
    return df


def clean_description(df):
    df["description"] = df["description"].fillna("no description")
    return df


def clean_superhost(df):
    ## If the superhost column contains the string "is a Superhost", assign 1 else 0
    df["superhost"] = df["superhost"].apply(lambda x: 1 if "is a Superhost" in x else 0)
    return df


def clean_response_rate(df):
    df["response_rate"] = df["response_rate"].apply(
        lambda x: x.replace("Host details\n", "").replace("Response rate: ", "")
    )
    df["response_time"] = df["response_rate"].apply(
        lambda x: x.split("%\n")[1] if "%\n" in x else "Unknown"
    )
    df["response_rate"] = df["response_rate"].apply(
        lambda x: x.split("%\n")[0] if "%\n" in x else "0"
    )
    return df


def clean_service_ratings(df):
    df["service_ratings"] = df["service_ratings"].apply(
        lambda x: x.replace("['Cleanliness\\n", "")
        .replace("', 'Accuracy\\n", "*/")
        .replace("', 'Check-in\\n", "*/")
        .replace("', 'Communication\\n", "*/")
        .replace("', 'Location\\n", "*/")
        .replace("', 'Value\\n", "*/")
        .replace("']", "")
        .replace("[]", "0.0*/0.0*/0.0*/0.0*/0.0*/0.0")
    )
    ## split the values into new columns
    df["cleanliness_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[0])
    df["accuracy_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[1])
    df["checkin_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[2])
    df["communication_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[3])
    df["location_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[4])
    df["value_rating"] = df["service_ratings"].apply(lambda x: x.split("*/")[5])
    ## Remove the service_ratings column
    df = df.drop("service_ratings", axis=1)
    return df


def clean_fee(df):
    df["fee"] = df["fee"].fillna("No fee found")

    def extract_fees(details):
        if not isinstance(details, str):
            return pd.Series(
                [None, None, None]
            )  # Return None if details is not a string

        # Extract per night price
        per_night_match = re.search(r"\$(\d+)\s*CAD\s*x", details)
        per_night = int(per_night_match.group(1)) if per_night_match else None

        # Extract cleaning fee
        cleaning_fee_match = re.search(
            r"Cleaning fee.*?\$(\d+)\s*CAD", details, re.DOTALL
        )
        cleaning_fee = int(cleaning_fee_match.group(1)) if cleaning_fee_match else None

        # Extract Airbnb service fee
        airbnb_fee_match = re.search(
            r"Airbnb service fee.*?\$(\d+)\s*CAD", details, re.DOTALL
        )
        airbnb_fee = int(airbnb_fee_match.group(1)) if airbnb_fee_match else None

        return pd.Series([per_night, cleaning_fee, airbnb_fee])

    # Apply the function and create new columns
    df[["Per_Night", "Cleaning_Fee", "Airbnb_Fee"]] = df["fee"].apply(extract_fees)
    ## drop the original column
    df = df.drop("fee", axis=1)
    ## fill per_night missing values with the discounted_price
    df["Per_Night"] = df["Per_Night"].fillna(df["discounted_price"])
    ## fill cleaning_fee missing values with 0
    df["Cleaning_Fee"] = df["Cleaning_Fee"].fillna(0)
    ## fill Airbnb_Fee missing values with 0
    df["Airbnb_Fee"] = df["Airbnb_Fee"].fillna(0)
    return df


def clean_host_description(df):
    ## host_description 1 if exists, 0 if "No host description found"
    df["host_description"] = df["host_description"].apply(
        lambda x: 0 if "No host description found" in x else 1
    )
    return df


def clean_house_rule(df):
    ## Extract the checkin and checkout time from the house_rules column
    df["checkin_time"] = df["house_rules"].apply(lambda x: x.split("\n")[1])
    df["checkout_time"] = df["house_rules"].apply(lambda x: x.split("\n")[2])
    df = df.drop("house_rules", axis=1)
    return df


def clean_amenities(df):
    ## Handle the word like "Air conditioning\\nAir conditioning"
    df["amenities"] = df["amenities"].apply(lambda x: re.sub(r"[\w\s]+\\n", "", x))
    ## Convert the string to list
    df["amenities"] = df["amenities"].apply(ast.literal_eval)

    # Function to separate available and unavailable amenities
    def split_amenities(amenities):
        # Separate unavailable amenities
        unavailable = [
            item.replace("Unavailable:", "")
            for item in amenities
            if item.startswith("Unavailable:")
        ]
        # Keep available amenities
        available = [item for item in amenities if not item.startswith("Unavailable:")]
        return pd.Series([available, unavailable])

    # Apply the function and create new columns
    df[["Available_Amenities", "Unavailable_Amenities"]] = df["amenities"].apply(
        split_amenities
    )
    ## Remove the amenities column
    df = df.drop("amenities", axis=1)
    return df


def clean_available_amenties(df):
    ## Clean the available amenities column since it contains a lot of common amenities with different names
    df["Available_Amenities"] = df["Available_Amenities"].apply(
        lambda x: [
            (
                amenity.replace(amenity, "HDTV")
                if "TV" in amenity
                else (
                    amenity.replace(amenity, "Wifi")
                    if "wifi" in amenity.lower()
                    else (
                        amenity.replace(amenity, "Body soap")
                        if "body soap" in amenity.lower()
                        else (
                            amenity.replace(amenity, "Coffee maker")
                            if "Coffee maker" in amenity
                            else (
                                amenity.replace(amenity, "Stove")
                                if "stove" in amenity
                                else (
                                    amenity.replace(amenity, "Shampoo")
                                    if "shampoo" in amenity
                                    else (
                                        amenity.replace(amenity, "Conditioner")
                                        if "conditioner" in amenity
                                        else (
                                            amenity.replace(amenity, "Oven")
                                            if "oven" in amenity
                                            else (
                                                amenity.replace(amenity, "Paid parking")
                                                if "Paid parking" in amenity
                                                else (
                                                    amenity.replace(
                                                        amenity, "Free parking"
                                                    )
                                                    if "Free parking" in amenity
                                                    else (
                                                        amenity.replace(
                                                            amenity, "EV charger"
                                                        )
                                                        if "EV charger" in amenity
                                                        else (
                                                            amenity.replace(
                                                                amenity, "Housekeeping"
                                                            )
                                                            if "Housekeeping" in amenity
                                                            else (
                                                                amenity.replace(
                                                                    amenity,
                                                                    "Refrigerator",
                                                                )
                                                                if "refrigerator"
                                                                in amenity
                                                                else (
                                                                    amenity.replace(
                                                                        amenity,
                                                                        "Hot tub",
                                                                    )
                                                                    if "hot tub"
                                                                    in amenity
                                                                    else (
                                                                        amenity.replace(
                                                                            amenity,
                                                                            "Sauna",
                                                                        )
                                                                        if "sauna"
                                                                        in amenity
                                                                        else (
                                                                            amenity.replace(
                                                                                amenity,
                                                                                "Backyard",
                                                                            )
                                                                            if "backyard"
                                                                            in amenity
                                                                            else (
                                                                                amenity.replace(
                                                                                    amenity,
                                                                                    "BBQ grill",
                                                                                )
                                                                                if "BBQ"
                                                                                in amenity
                                                                                else (
                                                                                    amenity.replace(
                                                                                        amenity,
                                                                                        "Fireplace",
                                                                                    )
                                                                                    if "fireplace"
                                                                                    in amenity
                                                                                    else (
                                                                                        amenity.replace(
                                                                                            amenity,
                                                                                            "Gym",
                                                                                        )
                                                                                        if "gym"
                                                                                        in amenity.lower()
                                                                                        else amenity
                                                                                    )
                                                                                )
                                                                            )
                                                                        )
                                                                    )
                                                                )
                                                            )
                                                        )
                                                    )
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
            for amenity in x
        ]
    )

    return df

In [ ]:
## Creating a pipeline to clean the data
def clean_data(df):
    cleaning_steps = [
        clean_basic_description,
        clean_prices,
        clean_ratings,
        clean_basic_info,
        clean_description,
        clean_superhost,
        clean_response_rate,
        clean_service_ratings,
        clean_fee,
        clean_host_description,
        clean_house_rule,
        clean_amenities,
        clean_available_amenties,
    ]
    
    for step in cleaning_steps:
        try:
            df = step(df)  # Apply each cleaning function
        except Exception as e:
            print(f"Error in {step.__name__}: {e}")  # Log error but continue

    return df


In [ ]:
df = clean_data(df)
df.to_csv("../data/cleaned_df.csv", index=False)